In [ ]:
# Пример решения с использованием статистического подхода - подсчет 
# совстречаемостей.

import pandas as pd
import numpy as np
import gc

from collections import Counter 

hist_data = pd.read_csv('hist_data.csv')

# соберем словарь встречаемостей - какие item_id покупались чаще с 
# каждым item_id 
tmp = (
    hist_data[['item_id', 'pav_order_id']]
    .sort_values(['item_id', 'pav_order_id'])
    .merge(hist_data[['item_id', 'pav_order_id']], how='left', on=['pav_order_id'], suffixes=('', '_left'))
)
tmp = tmp[tmp['item_id'] != tmp['item_id_left']].copy()
tmp1 = tmp.groupby(['item_id'])['item_id_left'].agg(lambda x: Counter(x).most_common(10))

most_freq_dict = {k: v for (k, v) in tmp1.iteritems()}

del tmp1, tmp
gc.collect()

test = pd.read_csv('test.csv')

# из списка кандидатов по совстречаемости удаляем повторяющиеся item_id, 
# сохраняя порядок
def get_unique_recs(recs: list, top_n: int) -> list:
    rec_dict = {}
    counter = 0
    for k, v in recs:
        if k not in rec_dict:
            rec_dict[k] = v
            counter += 1
        if counter == top_n:
            break
    return list(rec_dict.keys())

def rec_by_item(item_id: int, most_freq_dict: dict) -> list:
    
    return most_freq_dict.get(item_id, None)

# для каждого item_id соберем top_n самых часто встречающихся item_id, 
# отсортируем по частоте и выберем уникальные
def rec_by_basket(basket: list, most_freq_dict: dict, top_n: int = 20) -> list:
    
    res = []
    for item in basket:
        recs = rec_by_item(item, most_freq_dict)
        if recs is not None:
            res += recs
    
    res = sorted(res, key=lambda x: x[1], reverse=True)
    
    return get_unique_recs(res, top_n)

pred = test.groupby(['pav_order_id'])['item_id'].agg([('basket', list)])
pred['preds'] = pred['basket'].map(lambda x: rec_by_basket(x, most_freq_dict=most_freq_dict))

pred['preds'].to_csv('pred.csv')